In [1]:
import sys
sys.path.append('/home/trukhinmaksim/src')

In [2]:
import numpy as np

In [3]:
from src.utils.DatabaseConnect import DatabaseConnect

# single machine setup (mongo is running here localy)
# "ip a" for ip address
MY_DATABASE_LINK = 'mongodb://10.22.51.99:27020/' #'mongodb://192.168.100.57:27020/'
# multiple mechine setup (mongo is running on another machine)
#MY_DATABASE_LINK = 'mongodb://192.168.43.78:27020/'

DatabaseConnect.DB_LINK = MY_DATABASE_LINK

projectsCollection = DatabaseConnect.mini_database.projects()
usersCollection = DatabaseConnect.mini_database.users()
print(projectsCollection)

Collection(Database(MongoClient(host=['10.22.51.99:27020'], document_class=dict, tz_aware=False, connect=True), 'mini_database'), 'projects')


In [4]:
from src.utils.CacheAdapter import JSONAdapter, JSONMultiFileAdapter, EXP_END_OF_DATA
from src.utils.DatasetManager import ProjectsDatasetManager
from src.utils.validators import projectDataIsSufficient

In [5]:
def flatternData(data : dict[str, list]) -> np.array(dict):
    # takes in data in form of dict, where each key is a user id and each value is a list of that user's projects
    # returns just flat list of these projects 
    result = []

    for projectsArray in data.values():
        for project in projectsArray:
            result.append(project)

    return result

In [6]:
# using adapter to load data from the cache files

class DataFeeder(ProjectsDatasetManager):
    def __init__(self, batchSize = 32):
        #super().__init__(batchSize, validator, adapter)
        self.batchSize = batchSize

    def feedFromCache(self, cacheAdapter):
        # generator function, that will use "JSONMultifileAdapter" to parse data from multiple cache file, collect projects into a list and feed to the model
        tempStorage = []
        #self.cacheAdapter = cacheAdapter

        i = 0
        while True:
            try:
                while len(tempStorage) >= self.batchSize:
                    yield tempStorage[:self.batchSize]
                    tempStorage = tempStorage[self.batchSize:]

                data = cacheAdapter.load(1) # load users one by one
                data = flatternData(data)
                tempStorage.extend(data)

                i += 1

            except EXP_END_OF_DATA:
                break

        yield tempStorage

def feedProjectsFromCache(manager, batchSize = 32):
    cacheFileName = "cache__31-03-2025__(sufficient)_{index}.json"
    tempStorage = []

    i = 0
    while True:
        try:
            while len(tempStorage) >= batchSize:
                yield tempStorage[:batchSize]
                tempStorage = tempStorage[batchSize:]

            manager.cacheAdapter.collectionName = cacheFileName.format(index = i)
            data = flatternData(manager.fromCache())
            tempStorage.extend(data)

            i += 1

        except EXP_END_OF_DATA:
            break

    yield tempStorage

In [24]:
adapter = JSONAdapter()
ProjectsDatasetManager.usersCollection = usersCollection
ProjectsDatasetManager.projectsCollection = projectsCollection
manager = ProjectsDatasetManager(25, validate = projectDataIsSufficient, cacheAdapter = adapter)

In [26]:
"""
manager.fromDB()
textData = manager.getTextOnly()
for text in flatternData(textData):
    print(text)
    #print(flatternData(data))
    #print(text)
"""
def feedTextData(manager, batchSize = 32):
    tempStorage = []

    i = 0
    while True:
        try:
            while len(tempStorage) >= batchSize:
                yield tempStorage[:batchSize]
                tempStorage = tempStorage[batchSize:]

            manager.fromDB()
            data = flatternData(manager.getTextOnly())
            tempStorage.extend(data)

            i += 1

        except EXP_END_OF_DATA:
            break

    yield tempStorage

i = 2
for textBatch in feedTextData(manager, 32):
    if i <= 0: break
    print(len(textBatch))
    for text in textBatch:
        print(text)
    i -= 1


Scanning user: github:Dev4ster
Scanning user: github:EsunR
Scanning user: github:guopeng1994
Scanning user: github:inspiredole
Scanning user: github:batuhansahan
Scanning user: github:kolyneh
Scanning user: github:2017398956
Scanning user: github:StupidManYI
Scanning user: github:ddx-510
Scanning user: github:muhamedzeema
Scanning user: github:patrickbattisti
Scanning user: github:BrunoVillanova
Scanning user: github:lkoehl
Scanning user: github:coldSeattle
Scanning user: github:princefishthrower
Scanning user: github:somay
Scanning user: github:gabrielmoncea
Scanning user: github:wds1204
Scanning user: github:mthuong
Scanning user: github:soywod
Scanning user: github:copiri-six
Scanning user: github:m-camacho
Scanning user: github:abishekaditya
Scanning user: github:hasanfd
Scanning user: github:jimbrig
32
DevFinder Repo Dev Finder Projeto Codenation.dev em parceria com SAS
React-Native-SAS #AceleraDev: React Native promovido pela plataforma de educação SAS  :building_construction:
wh

In [17]:

i = 2
for batch in feedProjectsFromCache(manager):
    if i == 0: break
    print(len(batch))
    for item in batch:
        print(item)
    i -= 1

cache__31-03-2025__(sufficient)_0.json
32
{'tokens': ['awesome', 'code', 'streamer', 'list', 'code', 'streamer', 'multiple', 'plataforms', 'like', 'twitch', 'youtube'], 'tags': ['github:lucasfloriani/awesome-code-streamers', 'awesome-code-streamers', '', 'awesome', 'awesome-list', 'lists', 'resources']}
{'tokens': ['smart', 'home', 'mqtt', 'experimental', 'wip', 'bridge', 'google', 'smart', 'home', 'mqtt'], 'tags': ['github:alvarowolfx/smart-home-mqtt', 'smart-home-mqtt', 'JavaScript']}
{'tokens': ['typeorm', 'seeding', 'delightful', 'way', 'seed', 'test', 'data', 'database'], 'tags': ['github:betiol/typeorm-seeding', 'typeorm-seeding', 'TypeScript']}
{'tokens': ['express', 'response', 'formatter', 'sparkle', 'better', 'way', 'format', 'express', 'response'], 'tags': ['github:betiol/express-response-formatter', 'express-response-formatter', 'TypeScript']}
{'tokens': ['desafio', 'rn', 'desafio', 'react', 'native'], 'tags': ['github:betiol/desafio-rn', 'desafio-rn', 'TypeScript']}
{'toke

In [27]:

import gensim
from gensim.models.doc2vec import TaggedDocument

In [28]:
class Model(gensim.models.doc2vec.Doc2Vec):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.feeder = None # feeder is an iterator(iterable function or class object), that will be run with "for" loop to extract batches of data

    def train(self):
        pass

    def assess(self):
        pass


In [ ]:
"""
documentsCorpus = []

for project in manager.data:
    documentsCorpus.append(TaggedDocument(words=project["tokens"], tags=project["tags"]))

documentsCorpus
"""

In [31]:
# creating model

VECTOR_SIZE = 100
EPOCH_NUMBER = 10
WORD_MIN_AMOUNT = 3
WINDOW_SIZE = 7

model = Model(vector_size = VECTOR_SIZE, window = WINDOW_SIZE, min_count = WORD_MIN_AMOUNT, epochs = EPOCH_NUMBER)
print(model.epochs)
#model.build_vocab(documentsCorpus)
#model.train(documentsCorpus, total_examples = model.corpus_count, epochs = model.epochs)

10


In [ ]:
vector = model.infer_vector(['awesome', 'code', 'streamer', 'list', 'code', 'streamer', 'multiple', 'plataforms', 'like', 'twitch', 'youtube'])
vector